In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import header
paths = header.setup_environment()
import platform
if platform.system()=='Darwin':
    INPUTDIR = '/Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3_Project_Documents/NASAprojects/201602_Rocket_Seismology/DATA/2022_DATA/WellData/MERGED'
else:
    INPUTDIR = os.path.join(paths['new_data'], '03_merge_inventories')
import libWellData as LLE
transducersDF = LLE.get_transducers_dataframe(paths)
display(transducersDF)

# Load in the summary of all files - raw 4 hourly data 
dfall2 = pd.read_csv(os.path.join(INPUTDIR, 'all2.csv'))
#display(dfall2['subdir'])

LLE.qc_dataframe(dfall2)

# Split in baro, 20Hz, 100Hz
all_dataframes = LLE.split_by_subdir(dfall2, verbose=True)
#print(all_dataframes)


In [ ]:
'''
print(dfdata)
for k in dfdata.keys():
    print(k)
    display(dfdata[k])
'''

In [ ]:
'''
# 10.2: Correct analog and digital air column transducers for calibration, elevation above water, and apply DC shift (no temperature correction)
dcshifts2 = {'AirPressureShallow':0.0, 'AirPressureDeep':-0.003058, '1226420':-14.423795, '1226429':-14.556290}
aircolumns = ['AirPressureShallow', 'AirPressureDeep', '1226420', '1226429']
dfbaro_dcshifted = LLE.correctBarometricData(dfdata['baro'], aircolumns[:2], transducersDF, temperatureCorrect=False, heightCorrect=True, dcshifts=dcshifts2)
df100hz_dcshifted = LLE.correctBarometricData(dfdata['100hz'], aircolumns[2:], transducersDF, temperatureCorrect=False, heightCorrect=True, dcshifts=dcshifts2)

# 10.4: Correct digital water column transducers for calibration and barometric pressure
correctedAllSensorsPSI = LLE.rawdf2psidf(df100hz_dcshifted, transducersDF, temperatureCorrect=False, airpressureCorrect=True, depthCorrect=False)
watercolumns = ['1226419', '1226421', '2151691', '2149882']
display(correctedAllSensorsPSI[watercolumns])

# 10.5: plot PSI
correctedAllSensorsPSI.plot(x='datetime', y=aircolumns[2:]+watercolumns, style='-', ylabel='PSI')

# 10.6: convert to water levels in meters
correctedAllSensorsMeters = LLE.psi2meters(correctedAllSensorsPSI, watercolumns)
correctedAllSensorsMeters.plot(x='datetime', y=watercolumns, style='-', ylabel='Meters')

# 10.7: convert to water levels in meters relative to the set depth measured by Steve Krupa
relativeAllSensorsMeters = LLE.relative_to_set_depth(correctedAllSensorsMeters, transducersDF, watercolumns)
relativeAllSensorsMeters.plot(x='datetime', y=watercolumns, style='-', ylabel='Meters')

# 10.8: estimate correct set depths from median of each, and shift by this amount
estimatedAllSensorsMeters = LLE.estimate_sensor_depths(correctedAllSensorsMeters, watercolumns)
estimatedAllSensorsMeters.plot(x='datetime', y=watercolumns, style='-', ylabel='Meters')   
'''

In [ ]:
'''
# RSAM simulation
import obspy

import numpy as np
def simulate_rsam(thisdf, watercolumns):
    st = obspy.Stream()
    sample_interval = 1
    for col in thisdf.columns:
        if col in watercolumns:
            print(col)
            tr = obspy.Trace(data=thisdf[col].to_numpy())
            #nan_positions = np.where(np.isnan(tr.data))
            tr.data = np.nan_to_num(tr.data, nan=0.0)
            tr.detrend('linear')
            tr.filter('highpass', freq=0.05)
            #tr.data[nan_positions] = np.nan
            thisdf[col] = abs(tr.data)
    #print(thisdf.columns)
    #thisdf.reset_index(inplace=True)
    display(thisdf)
    #resampleddf = thisdf.resample('10s', on='datetime').median()
    resampleddf = thisdf.resample(f'{sample_interval}s', on='datetime').median()
    print(resampleddf)
    #for col in resampleddf.columns:
    #    resampleddf[col] = resampleddf[col]-resampleddf[col][0]
    resampleddf.plot(kind='line')
    plt.show()
    for col in resampleddf:
        tr2 = obspy.Trace(data=resampleddf[col].to_numpy())
        tr2.data = np.nan_to_num(tr2.data, nan=0.0)
        tr2.id = f'{col[0:2]}.{col[2:7]}.{col[7:9]}.{col[9:]}'
        tr2.stats.delta = float(sample_interval)
        tr2.stats.starttime=obspy.UTCDateTime(resampleddf.index[0])
        st.append(tr2)
    st.plot(equal_scale=False);
    print(st)
    return st
rsamst = simulate_rsam(estimatedAllSensorsMeters, watercolumns)
'''

In [ ]:
'''
#st.filter('bandpass', freqmin=10, freqmax=20)  # optional prefiltering
print(st)
from obspy.signal.trigger import coincidence_trigger

st2 = st.copy()

trigs = coincidence_trigger("recstalta", 2, 1, st2, int(len(st2)/2), sta=sample_interval*5, lta=sample_interval*100)
for thistrig in trigs:
    display(thistrig)
    noisewindow = st2.copy().trim(starttime=thistrig['time']-60, endtime=thistrig['time']-10)
    for tr in noisewindow:
        if not np.all(tr.data):
            thistrig['trace_ids'].remove(tr.id)
    if len(thistrig['trace_ids'])>len(st2)/2:
        st3 = st2.copy().trim(starttime=thistrig['time']-60, endtime=thistrig['time']+60+thistrig['duration'])
        st3.plot(equal_scale=False);
'''